# Marine Biodiversity and Environmental Data Exploration with `mlcroissant`
This notebook provides a guide for loading and exploring the Marine Biodiversity and Environmental dataset using the `mlcroissant` library.

### Dataset Source
The dataset source is provided via a Croissant schema URL.

In [ ]:
# Ensure `mlcroissant` library is installed
!pip install mlcroissant

## 1. Data Loading
Load metadata and records from the dataset using `mlcroissant`.

In [ ]:
import mlcroissant as mlc
import pandas as pd

# Define the dataset URL
url = 'https://cdn.dev.senscience.cloud/portals/10.82843/pm80-mh77/fair2.json'

# Load the dataset metadata
dataset = mlc.Dataset(url)
metadata = dataset.metadata
print(f"{metadata.name}: {metadata.description}")

## 2. Data Overview
Review available record sets, fields, and their IDs.

In [ ]:
record_sets = metadata.record_set
for rs in record_sets:
    print(f"Record Set ID: {rs['@id']}")
    for field in rs['field']:
        print(f"  Field: {field['name']} (ID: {field['@id']})")

## 3. Data Extraction
Load data from a specific record set into a DataFrame for analysis. Use the record set and field `@id`s from the overview.

In [ ]:
# Extract data from each record set
record_sets_ids = [
    'https://senscience.ai/frontiers/borja/README.csv',
    'https://senscience.ai/frontiers/borja/METADATA.csv'
]
dataframes = {}

for record_set_id in record_sets_ids:
    records = list(dataset.records(record_set=record_set_id))
    dataframes[record_set_id] = pd.DataFrame(records)

print(dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'].columns.tolist())
dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'].head()

## 4. Exploratory Data Analysis (EDA)
Apply common data processing steps, such as filtering records based on specific criteria, normalizing numeric fields, and categorizing data. This section should include operations like removing outliers, transforming data distributions, or grouping data by key attributes to prepare it for further analysis.

In [ ]:
# Select a numeric field for analysis
numeric_field = 'https://api.dev.senscience.cloud/frontiers/8704cfcf-0667-4eaf-b0b1-679597df683f-16/ceb7cb36-2f56-4ee2-ba06-46cba6d79804'

threshold = 43.35
filtered_df = dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'][
    dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'][numeric_field] > threshold
]
print(f"Filtered records with {numeric_field} > {threshold}:")
print(filtered_df.head())

filtered_df[f"{numeric_field}_normalized"] = (
    filtered_df[numeric_field] - filtered_df[numeric_field].mean()
) / filtered_df[numeric_field].std()
print(f"Normalized {numeric_field} for filtered records:")
print(filtered_df[[numeric_field, f"{numeric_field}_normalized"]].head())

group_field = 'https://api.dev.senscience.cloud/frontiers/8704cfcf-0667-4eaf-b0b1-679597df683f-16/2115867c-3fa3-405d-9d94-ec8f455b19dc'
if group_field in dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'].columns:
    grouped_df = filtered_df.groupby(group_field).mean()
    print(f"Grouped data by {group_field}:")
    print(grouped_df.head())

## 5. Visualization
Visualize data distributions or relationships between fields in the dataset.

In [ ]:
import matplotlib.pyplot as plt

# Visualize the distribution of a numeric field
plt.hist(
    dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'][numeric_field],
    bins=30
)
plt.title('Distribution of Decimal Latitude')
plt.xlabel('Latitude')
plt.ylabel('Frequency')
plt.show()

## 6. Conclusion
Summarize key findings and observations from the dataset exploration.

Through this analysis, we have loaded and explored the Marine Biodiversity and Environmental dataset. Key fields were identified, and initial filtering and normalization of data were conducted. Visualization of latitude distribution provided insights into sampling site locations. Further analysis could delve into temporal trends or correlations between environmental variables.